In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import pickle
import sklearn.datasets as datasets
import sklearn.metrics as mt
import seaborn as sns

In [ ]:
plt.rcParams['figure.figsize'] = (10, 6)
plt.style.use('dark_background')

# Dataset

In [ ]:
x, y = datasets.make_moons(n_samples = 500, noise = 0.05)
x.shape, y.shape

In [ ]:
pd.DataFrame({'x_1': x[:,0], 'x_2': x[:,1], 'y': y})

In [ ]:
unique = np.unique(y, return_counts=True)
for label, qt_label in zip(unique[0], unique[1]):
    print(f'Label: {label}\t Counts: {qt_label}')

In [ ]:
plt.scatter(x[:,0], x[:,1], c = y, s = 50, alpha=0.5, cmap= 'cool')

# Model
- Weights and Bias Inicialization
- Feedfoward
- Loss calculation
- Backpropagation
- Fit

In [ ]:
class NnModel:
    def __init__(self, x: np.ndarray, y: np.ndarray, hidden_neurons: int = 10, output_neurons: int = 2):
        self.x = x
        self.y = y
        self.hidden_neurons = hidden_neurons
        self.output_neurons = output_neurons
        self.input_neurons = self.x.shape[1]
        
        # Weights and Bias Inicialization
        # Xavier Inialization -> equal variation in all layers
        
        self.W1 = np.random.randn(self.input_neurons, self.hidden_neurons) / np.sqrt(self.input_neurons)
        self.B1 = np.zeros((1, self.hidden_neurons))
        
        self.W2 = np.random.randn(self.hidden_neurons, self.output_neurons) / np.sqrt(self.hidden_neurons)
        self.B2 = np.zeros((1, self.output_neurons))

        self.model_dict = {'W1': self.W1, 'B1': self.B1, 'W2': self.W2, 'B2': self.B2}
        
        #Z and activation function (1)
        self.z1 = 0
        self.f1 = 0
    
    def foward(self, x: np.ndarray) -> np.ndarray:
        # Line Equation Z (1)
        self.z1 = x.dot(self.W1) + self.B1
        
        # Activation Function (1)
        self.f1 = np.tanh(self.z1)
        
        # Line Equation Z (2)
        z2 = self.f1.dot(self.W2) + self.B2
        
        # Softmax
        exp_values = np.exp(z2)
        softmax = exp_values / np.sum(exp_values, axis = 1, keepdims = True)
        
        return softmax
    
    def loss(self, softmax):
        # Cross Entropy
        predictions = np.zeros(self.y.shape[0])
        
        for i, correct_index in enumerate(self.y):
            predicted = softmax[i][correct_index]
            predictions[i] = predicted
        
        log_prob = -np.log(predicted)
        
        return log_prob / self.y.shape[0]
    
    def backpropagation(self, softmax: np.ndarray, learning_rate: float) -> None :
        delta2 = np.copy(softmax)
        
        delta2[range(self.x.shape[0]), self.y] -= 1
        
        dW2 = (self.f1.T).dot(delta2)
        dB2 = np.sum(delta2, axis = 0, keepdims = True)
        
        delta1 = delta2.dot(self.W2.T)*(1-np.power(np.tanh(self.z1), 2))
        dW1 = (self.x.T).dot(delta1)
        dB1 = np.sum(delta1, axis = 0, keepdims = True)
        
        # Weight update
        self.W1 += - learning_rate * dW1
        self.W2 += - learning_rate * dW2
        self.B1 += - learning_rate * dB1
        self.B2 += - learning_rate * dB2
    
    # NOT NEEDED - functions shows plots of the learning
    def show_plot(self, predictions):
        if self.x.shape[1] == 2:
            plt.scatter(self.x[:,0], self.x[:,1], s=50,c=predictions, cmap='cool', alpha=0.7)
            plt.show()
        elif self.x.shape[1] == 3:
            ax = plt.axes(projection='3d')
            ax.scatter3D(self.x[:,0], self.x[:,1], self.x[:,2], s=50,c=predictions, cmap='cool', alpha=0.7)
        
    def fit(self, epochs: int, lr: float):
        
        for epoch in range(epochs):
           outputs = self.foward(self.x)
           loss = self.loss(outputs)
           self.backpropagation(outputs, lr)
           
           # Accuracy
           prediction = np.argmax(outputs, axis=1)
           correct = (prediction == self.y).sum()
           accuracy = correct/self.y.shape[0]
           
           if int((epoch+1) % (epochs/10)) == 0:
               print(f'Epoch: [{epoch+1} / {epochs}] Accuracy: {accuracy:.3f} Loss: {loss.item():.4f}')
               self.show_plot(prediction)
        
        return prediction

# Tests

In [ ]:
hidden_neurons = 10
output_neurons = 2
learning_rate = 0.001
epochs = 100

model = NnModel(x, y, hidden_neurons, output_neurons)
result = model.fit(epochs, learning_rate)

# Cluster dataset

In [ ]:
x_, y_ = datasets.make_blobs(n_samples = 400, n_features=2, centers=4, random_state=10, cluster_std=1.5, shuffle=True)

#plt.scatter(x_[:,0], x_[:,1], s=50,c=y_, cmap='cool', alpha=0.7)

hidden_neurons = 8
output_neurons = 4
learning_rate = 0.001
epochs = 100

model = NnModel(x_, y_, hidden_neurons, output_neurons)
result = model.fit(epochs, learning_rate)

# 3D

In [ ]:
x_, y_ = datasets.make_blobs(n_samples = 400, n_features=3, centers=4, random_state=50, cluster_std=0.9, shuffle=True)

hidden_neurons = 10
output_neurons = 4
learning_rate = 0.001
epochs = 100

modelo_3d = NnModel(x_, y_, hidden_neurons, output_neurons)
resultado_3d = modelo_3d.fit(epochs, learning_rate)

# New Dataframe

In [ ]:
df = datasets.load_breast_cancer(as_frame=True)

x_bc = df.data.to_numpy()[:, 4:9]
y_bc = df.target.to_numpy()

bc = NnModel(x_bc, y_bc, 5, 2)
result_bc = bc.fit(2000, 0.001)

confusion_matrix = mt.confusion_matrix(y_bc, result_bc)
sns.heatmap(confusion_matrix, annot=True, cmap='coolwarm', fmt='.0f')

# Save Models and Load

In [ ]:
with open('modelo_treinado.pickle', 'wb') as file:
    pickle.dump(bc, file)

In [ ]:
with open('modelo_treinado.pickle', 'rb') as file:
    modelo_carregado = pickle.load(file)

modelo_carregado.foward(np.array([0.085,0.064,0.01,0.0099, 0.078])).argmax()